<a href="https://colab.research.google.com/github/manishiitg/ML_Experments/blob/master/ml/ski_lean_ml_classifier_naive_bayes_random_forest_svc_sgd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Trying to different ML classifiers like naive bayes, random forest, svc, sgd for multi label classification

In [0]:
%matplotlib inline

import pandas as pd
import numpy as np

import spacy 
nlp = spacy.load("en_core_web_sm")

from sklearn.datasets import fetch_20newsgroups

from gensim.models import KeyedVectors

from sklearn.cluster import KMeans;
from sklearn.neighbors import KDTree;

from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt;
from itertools import cycle;
import matplotlib.pyplot as plt;
from sklearn.model_selection import train_test_split


from sklearn.feature_extraction.text import TfidfVectorizer


from collections import defaultdict


In [0]:
def normalize(comment, lowercase=True, remove_stopwords=True):
    if lowercase:
        comment = comment.lower()
    lines = comment.splitlines()
    lines = [x.strip(' ') for x in lines]
    lines = [x.replace('"', '') for x in lines]
    lines = [x.replace('\\"', '') for x in lines]
    lines = [x.replace(u'\xa0', u'') for x in lines]
    comment = " ".join(lines)
    doc = nlp(comment)

    # for token in doc:
    #   print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
    #     token.shape_, token.is_alpha, token.is_stop)

    words = [token for token in doc if token.is_stop !=
             True and token.is_punct != True]
    # return " ".join(words)
    lemmatized = list()
    for word in words:
        lemma = word.lemma_.strip()
        if lemma:
            lemmatized.append(lemma)
    return lemmatized

In [3]:
import os

def writetofile(dir, filename, data):
    if not os.path.exists(dir):
        os.makedirs(dir)
    f = os.path.join(dir,str(filename))

    with open(f, 'wb') as the_file:
      the_file.write(data)

news = fetch_20newsgroups(subset="train", categories=[ 'comp.graphics','comp.os.ms-windows.misc','comp.sys.ibm.pc.hardware','comp.sys.mac.hardware','comp.windows.x'])


clean_data  = []

# print(news.keys())

# print(news["filenames"][:10])

# print(len(news["data"][:1000]))

# print(news["target_names"][:50])
# print(news["target"][:10])


max_limit = 4000

targets = news["target"][:max_limit]
filenames = news["filenames"][:max_limit]
news = news["data"][:max_limit]

dir = "news_group_cleaned"

print("cleaning data")
for i, row in enumerate(news):
  filename = filenames[i]
  filename = filename[(filename.rfind('/'))+1:]
  if os.path.exists(filename):
    with open(os.path.join(dir, filename), 'r') as content_file:
      data = content_file.read()
      cleaned = data.split(" ")
  else:
    cleaned = normalize(row)
    writetofile(dir, filename, " ".join(cleaned).encode("utf-8"))
    
  
  clean_data.append(cleaned)

print("data cleaned")

# print(clean_data[10])

cleaning data
data cleaned


In [4]:
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

model = gensim.models.Word2Vec(clean_data)


# model = KeyedVectors.load_word2vec_format('wiki-news-300d-1M.vec')

#https://stackoverflow.com/questions/45159693/word2vec-models-consist-of-characters-instead-of-words

2019-11-08 06:06:24,021 : INFO : collecting all words and their counts
2019-11-08 06:06:24,022 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-11-08 06:06:24,140 : INFO : collected 66979 word types from a corpus of 470869 raw words and 2936 sentences
2019-11-08 06:06:24,141 : INFO : Loading a fresh vocabulary
2019-11-08 06:06:24,179 : INFO : effective_min_count=5 retains 7758 unique words (11% of original 66979, drops 59221)
2019-11-08 06:06:24,180 : INFO : effective_min_count=5 leaves 392121 word corpus (83% of original 470869, drops 78748)
2019-11-08 06:06:24,207 : INFO : deleting the raw counts dictionary of 66979 items
2019-11-08 06:06:24,211 : INFO : sample=0.001 downsamples 33 most-common words
2019-11-08 06:06:24,212 : INFO : downsampling leaves estimated 326228 word corpus (83.2% of prior 392121)
2019-11-08 06:06:24,237 : INFO : estimated required memory for 7758 words and 100 dimensions: 10085400 bytes
2019-11-08 06:06:24,238 : INFO : resetting 

In [0]:
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = 100

    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        #https://stackoverflow.com/a/5900634
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

w2v = dict(zip(model.wv.index2word, model.wv.vectors))

tf = TfidfEmbeddingVectorizer(w2v)
tf.fit(clean_data, targets)
embeddings = tf.transform(clean_data)   



In [6]:
print(len(embeddings))
print(len(targets))

X_train, X_test, Y_train, Y_test = train_test_split(embeddings, targets, test_size=0.33, random_state=42)

2936
2936


In [7]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train, Y_train)

pred = clf.predict(X_test)

clf.score(X_test, Y_test)

0.4282765737874097

In [8]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
clf.fit(X_train, Y_train) 

pred = clf.predict(X_test)

clf.score(X_test, Y_test)

0.4674922600619195

In [9]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, Y_train)

pred = clf.predict(X_test)

clf.score(X_test, Y_test)

ValueError: ignored

In [10]:
from sklearn.svm import SVC

clf = SVC(kernel='linear')
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
clf.score(X_test, Y_test)

0.6749226006191951

In [11]:
from sklearn.model_selection import GridSearchCV
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
gs_clf = GridSearchCV(clf, parameters, cv=5, iid=False, n_jobs=-1)

gs_clf = gs_clf.fit(X_train[:400], Y_train[:400])

print(gs_clf.best_score_)
print(gs_clf.best_params_)

for param_name in sorted(parameters.keys()):
  print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

0.6075988582291968
{'C': 10, 'kernel': 'linear'}
C: 10
kernel: 'linear'


In [12]:
from sklearn.linear_model import SGDClassifier

# clf = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)
clf = SGDClassifier()
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
clf.score(X_test, Y_test)

0.6398348813209495

more important we can see how tune parameters one a model is chosen using

https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html#parameter-tuning-using-grid-search

also look at classification report 


OneVsRest strategy can be used for multi-label learning, where a classifier is used to predict multiple labels for instance. Naive Bayes supports multi-class, but we are in a multi-label scenario, therefore, we wrap Naive Bayes in the OneVsRestClassifier.